# Dependencies

In [ ]:
from langchain_community.document_loaders import PyPDFLoader,TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter 
from openai import OpenAI
import requests

from langchain_openai import ChatOpenAI
from langchain.chains import LLMChain
from pinecone import Pinecone, ServerlessSpec
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate


from langchain.agents import initialize_agent, Tool
from langchain.chains import ConversationChain
from langchain.memory import ConversationBufferMemory
from langchain.schema import HumanMessage, AIMessage
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

from langchain.agents.format_scratchpad.openai_tools import (
    format_to_openai_tool_messages,
)
from langchain.agents.output_parsers.openai_tools import OpenAIToolsAgentOutputParser
from langchain.agents import AgentExecutor

from langchain_core.tools import tool
from pydantic.v1 import BaseModel, Field
from openai import OpenAI

from pydantic import BaseModel, Field, ValidationError
from typing import List, Union, Dict, Optional
from pinecone import Index

import validators
import requests
import os
import uuid

from dotenv import load_dotenv
load_dotenv()

import os
import sys
import time

OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
PINECONE_API_KEY = os.getenv("PINECONE_API_KEY")
PINECONE_API_ENV = os.getenv("PINECONE_API_ENV")

For the answer generation, let us now set the system instructions for the LLM model, which would be used to generate the answers.

In [13]:
system_instructions = """
    You are a specialised AI helper, but you must realise that if a query comes to you, it has already crossed the knowledge bank 
    and still the user is asking, means they clearly need a very simple and easy to digest explaination. Try to help them with examples 
    and annecdotes to make sure they understand. Although the questions you would be getting would be of the CS concept, but 
    try to dumb them down for the user.
"""

# Assistant API 
Now, we have an assistant ready to be used, which can be used to go through the query, and then generate the answers. We will be using hitting the assistant with an API call, and then getting the answers from the assistant.

In [14]:
openAI_client = OpenAI(api_key=OPENAI_API_KEY)
assistant_id = os.getenv("FALLBACK_OPENAI_ASSISTANT_ID")

Now, we will be building the function to answer the queries from the assistant, which would be an asynchronus function on a thread as we aim to fetch the answer from the assistant.

In [15]:
thread = openAI_client.beta.threads.create()

def fetch_answer(final_prompt):
    openAI_client.beta.threads.messages.create(
        thread_id=thread.id,
        role="user",
        content= [{
            "type" : "text",
            "text" : final_prompt
        }]
    )
            
    run = openAI_client.beta.threads.runs.create(
        thread_id=thread.id,
        assistant_id=assistant_id,
    )
    
    while True:
        run = openAI_client.beta.threads.runs.retrieve(
            thread_id=thread.id,
            run_id=run.id
        )
        if run.status == 'completed':
            messages = openAI_client.beta.threads.messages.list(
                thread_id=thread.id
            )
            return messages.data[0].content[0].text.value
        elif run.status in ['failed', 'cancelled', 'expired']:
            raise Exception(f"Run ended with status: {run.status}")
        time.sleep(1)

# Agent
Now, we will have to make the tools for the agent and make it ready to be used. We will be using the agent to fetch the user query (ideally, this would be coming in from the supervisor), and then we will be using the agent to fetch the answers from the assistant (eventually to be sent back to the supervisor).

In [ ]:
@tool
def answer_query(query):
    """
        Find relevant documents for a given query and userID.
        
        Args:
        - query: The search query
       
        Returns:
        Final text answer from the LLM, given the relevant documents.
    """
        
    return fetch_answer(query)
    

Now that we have the tool ready, we will be making the prompt for the agent. This aims to clearly tell the assisant what has to be done and what is the query.

In [ ]:
main_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are an assistant who can handle queries of students, however you are only acitvated in the case they do not understand from the main sources. Hence, YOU MUST TELL AT THE START OF YOUR ANSWER CLEARLY THAT THIS IS AI GENERATED (even a tag like [AI GENERATED] telling the user with a warning will be optimal)...",
        ),
        ("user", "{input}"),
        MessagesPlaceholder(variable_name="agent_scratchpad"),
    ]
)

Let us now initialise the LLM instance from langchain and then use the tools above and bind them to the agent, which would be used to fetch the answers.

In [18]:
llm = ChatOpenAI(
    api_key = OPENAI_API_KEY,
    model = "gpt-4o",
    temperature=0.2
)

In [19]:
tools = [answer_query]
llm_with_tools = llm.bind_tools(tools)

Now, this function creates an agent pipeline in a step-by-step flow, processing input data and returning a meaningful output. How the Pipeline Works Together : 
- Input Preparation: The user’s query (x["input"]) and intermediate steps are extracted and formatted.
- Main Prompt Application: The instructions for the agent (e.g., "You are a helpful assistant") are added to guide its behavior.
- Processing by LLM: The LLM processes the input and, if needed, interacts with tools to generate an appropriate response.
- Output Parsing: The raw response from the LLM is cleaned up for easy understanding by the user.

In [20]:
agent = (
    {
        "input": lambda x: x["input"],
        "agent_scratchpad": lambda x: format_to_openai_tool_messages(
            x["intermediate_steps"]
        ),
    }
    | main_prompt
    | llm_with_tools
    | OpenAIToolsAgentOutputParser()
)

With this, we can execute the function and see how the agent works.

In [21]:
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)
list(agent_executor.stream({"input": "I am still having issues with my NEXT JS AUTH, ughh"}))



> Entering new None chain...

Invoking: `answer_query` with `{'query': 'I am still having issues with my NEXT JS AUTH, ughh'}`


No worries, let's try to break it down and make it as simple as possible! Authentication in Next.js can sometimes be a bit tricky, but once you get the hang of it, it becomes much easier. Let's go through the basics.

### What is Next.js Auth?

Next.js is a React framework that allows you to build server-rendered applications easily. Authentication (auth) is the process of verifying who a user is. In Next.js, you often use libraries like `next-auth` to handle this process.

### How Does Authentication Work?

1. **User Login**: The user enters their credentials (like username and password) on your website.
2. **Verification**: These credentials are sent to a server to verify if they are correct.
3. **Session Creation**: If the credentials are correct, a session is created. Think of a session like a temporary pass that allows the user to access certain parts 

[{'actions': [ToolAgentAction(tool='answer_query', tool_input={'query': 'I am still having issues with my NEXT JS AUTH, ughh'}, log="\nInvoking: `answer_query` with `{'query': 'I am still having issues with my NEXT JS AUTH, ughh'}`\n\n\n", message_log=[AIMessageChunk(content='', additional_kwargs={'tool_calls': [{'index': 0, 'id': 'call_4opSaOaKFYX4iB2joAXEuza8', 'function': {'arguments': '{"query":"I am still having issues with my NEXT JS AUTH, ughh"}', 'name': 'answer_query'}, 'type': 'function'}]}, response_metadata={'finish_reason': 'tool_calls', 'model_name': 'gpt-4o-2024-08-06', 'system_fingerprint': 'fp_50cad350e4'}, id='run-e41c7496-d63e-4f33-9ecb-c669bd032ea6', tool_calls=[{'name': 'answer_query', 'args': {'query': 'I am still having issues with my NEXT JS AUTH, ughh'}, 'id': 'call_4opSaOaKFYX4iB2joAXEuza8', 'type': 'tool_call'}], tool_call_chunks=[{'name': 'answer_query', 'args': '{"query":"I am still having issues with my NEXT JS AUTH, ughh"}', 'id': 'call_4opSaOaKFYX4iB2joA

In [22]:
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)
list(agent_executor.stream({"input": "I am so confused about how JWT works..."}))



> Entering new None chain...

Invoking: `answer_query` with `{'query': 'how JWT works'}`


Alright, let's dive into the world of JWTs (JSON Web Tokens) in a way that's easy to understand!

### What is a JWT?

A JWT is like a digital ID card that you carry around to prove who you are. It's a compact, URL-safe way of representing claims between two parties. These claims can be anything, but they're usually about the user and their permissions.

### How Does a JWT Work?

1. **Structure**: A JWT is made up of three parts:
   - **Header**: Contains metadata about the token, like the type of token (JWT) and the signing algorithm used (e.g., HMAC SHA256).
   - **Payload**: This is the part where the actual data (claims) is stored. For example, user ID, username, and roles.
   - **Signature**: This is created by taking the header and payload, encoding them, and then signing them with a secret key. This ensures that the token hasn't been tampered with.

   When you put these parts together, t

[{'actions': [ToolAgentAction(tool='answer_query', tool_input={'query': 'how JWT works'}, log="\nInvoking: `answer_query` with `{'query': 'how JWT works'}`\n\n\n", message_log=[AIMessageChunk(content='', additional_kwargs={'tool_calls': [{'index': 0, 'id': 'call_vVUB5IvkgqZIGct330HoCxuC', 'function': {'arguments': '{"query":"how JWT works"}', 'name': 'answer_query'}, 'type': 'function'}]}, response_metadata={'finish_reason': 'tool_calls', 'model_name': 'gpt-4o-2024-08-06', 'system_fingerprint': 'fp_50cad350e4'}, id='run-7f26d284-1434-4bf2-964b-d0859d1b53eb', tool_calls=[{'name': 'answer_query', 'args': {'query': 'how JWT works'}, 'id': 'call_vVUB5IvkgqZIGct330HoCxuC', 'type': 'tool_call'}], tool_call_chunks=[{'name': 'answer_query', 'args': '{"query":"how JWT works"}', 'id': 'call_vVUB5IvkgqZIGct330HoCxuC', 'index': 0, 'type': 'tool_call_chunk'}])], tool_call_id='call_vVUB5IvkgqZIGct330HoCxuC')],
  'messages': [AIMessageChunk(content='', additional_kwargs={'tool_calls': [{'index': 0, '

With this, we now have an agent which will be answering the core question asked by the user, in a very easy and comprehensible way.